In [6]:
import great_expectations as gx
import pandas as pd
import warnings
warnings.filterwarnings("ignore", message="`result_format` configured at the Validator-level*")

# Load the data
df = pd.read_csv("./data/sebank_customers_with_accounts.csv")
df.index += 2

# Create the ephemeral GX context
context = gx.get_context()

# Add a pandas datasource
data_source = context.data_sources.add_pandas(name="pandas")

# Add a dataframe asset
data_asset = data_source.add_dataframe_asset(name="accounts_data")

# Define the batch (entire DataFrame)
batch_definition = data_asset.add_batch_definition_whole_dataframe(name="batch_def")
batch = batch_definition.get_batch(batch_parameters={"dataframe": df})

# Create the expectation suite with a name
suite = gx.core.expectation_suite.ExpectationSuite(name="accounts_suite")

# Get the validator using the suite
validator = context.get_validator(batch=batch, expectation_suite=suite)

# Add expectations
# Validate Customer column
validator.expect_column_values_to_not_be_null("Customer")

# Validate Address column
validator.expect_column_values_to_not_be_null("Address")

"""
# Validate PostalCode column
validator.expect_column_values_to_match_regex("PostalCode", r"^\d{5}$")
"""
validator.expect_column_values_to_not_be_null("Phone")
validator.expect_column_values_to_match_regex("Phone", r"^(\+46\s?\(0\)|0)?\d{1,4}[- ]\d{2,4}([ -]\d{2,4}){1,3}$")
validator.expect_column_values_to_be_unique("Phone")


validator.expect_column_values_to_not_be_null("Personnummer")
validator.expect_column_values_to_match_regex("Personnummer", r"^\d{6,8}[-]?\d{4}$")


validator.expect_column_values_to_not_be_null("BankAccount")
validator.expect_column_values_to_match_regex("BankAccount", r"^SE8902")
validator.expect_column_values_to_be_unique("BankAccount")

# Validate
results = validator.validate()

# Print results
print(results)

<>:36: DeprecationWarning: invalid escape sequence '\d'
<>:36: DeprecationWarning: invalid escape sequence '\d'
C:\Users\fintn\AppData\Local\Temp\ipykernel_2740\189020841.py:36: DeprecationWarning: invalid escape sequence '\d'
  """

Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 158.80it/s]

Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 157.62it/s]

Calculating Metrics: 100%|██████████| 6/6 [00:00<00:00, 204.32it/s]

Calculating Metrics:  38%|███▊      | 3/8 [00:00<00:00, 186.19it/s]C:\Users\fintn\AppData\Local\Programs\Python\Python311\Lib\site-packages\great_expectations\expectations\metrics\column_map_metrics\column_values_match_regex.py:41: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return column.astype(str).str.contains(regex)

Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 183.72it/s]

Calculating Metrics: 100%|██████████| 8/8 [00:00<00:00, 223.74it/s]

Calcula

{
  "success": false,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "pandas-accounts_data",
          "column": "Customer"
        },
        "meta": {}
      },
      "result": {
        "element_count": 1000,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexpected_list": []
      },
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "pandas-accounts_data",
          "column": "Address"
        },
        "meta": {}
      },
      "result": {
        "element_count": 1000,
        "unexpected_count": 0,
        "unexpected_percent": 0.0,
        "partial_unexp